In [1]:
import numpy as np
import pandas as pd
from math import pi
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, MinMaxScaler, StandardScaler
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.models import Model, load_model
from keras.layers import GRU, Dense, Dropout, concatenate, Input
from keras.optimizers import RMSprop
from random import shuffle
import warnings
import holidays
from datetime import timedelta
np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [2]:
data =pd.read_csv('./actuals.csv')


train_cut = int(len(data) * 0.8)
validate_cut = int(len(data) * 0.9)
test_cut = int(len(data))

In [3]:
X, y = data, data['rides']
X.head()

,date,sunrise,icon,precip_prob,temperature,humidity,wind_speed,rides
0,2013-06-01 00:00:00,0,clear,0.01,77.65,0.61,2.06,152
1,2013-06-01 01:00:00,0,clear,0.01,75.62,0.67,1.93,102
2,2013-06-01 02:00:00,0,clear,0.01,74.72,0.70,2.31,67
3,2013-06-01 03:00:00,0,clear,0.01,73.32,0.76,2.16,41
4,2013-06-01 04:00:00,0,clear,0.01,72.42,0.79,1.93,16


In [4]:
y.head()

0    152
1    102
2     67
3     41
4     16
Name: rides, dtype: int64

<h2>Define Custom Transformers and Pipelines</h2>

In [5]:
class HolidaySelector(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        hd = [date for date, name in holidays.US(years=[2013, 2014, 2015, 2016, 2017, 2018]).items()
                        if name.startswith(("New Year's Day", "Washington's Birthday", "Memorial Day", "Independence Date",
                        "Labor Day", "Thanksgiving", "Christmas Day"))]
        hd_eve = [day - timedelta(days=1) for day in hd]
        hd.extend(hd_eve)
        self.h = [str(date) for date in hd]
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X[['date']].applymap(lambda x: int(pd.to_datetime(x).strftime('%Y-%m-%d') in self.h))
    
class DateTimeExtractor(BaseEstimator, TransformerMixin):
    
    def __init__(self, extract):
        self.extract = extract
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        return X[['date']].applymap(lambda x: float(getattr(pd.to_datetime(x), self.extract)))

class ColumnSelector(BaseEstimator, TransformerMixin):

    def __init__(self, columns=None):
        self.columns = columns

    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        return X.loc[:, self.columns]


In [6]:
all_pipeline = Pipeline([
    ('union', FeatureUnion([
        ('onehot_pipeline', Pipeline([
            ('onehot_union', FeatureUnion([
                ('hour', DateTimeExtractor('hour')),
                ('month', DateTimeExtractor('month')), 
                ('dayofweek', DateTimeExtractor('dayofweek')), 
                ('categories', ColumnSelector(['sunrise', 'icon'])),
                ('holiday', HolidaySelector()),
            ])),
            ('onehot_encoder', OneHotEncoder(sparse=False))
        ])),
        ('float_pipeline', Pipeline([
            ('float_union', FeatureUnion([
                ('year', DateTimeExtractor('year')),
                ('floats', ColumnSelector(['precip_prob', 'temperature', 'humidity', 'wind_speed'])),
            ])),
            ('scaler', StandardScaler())
        ]))
    ]))
])

time_pipeline = Pipeline([
    ('union', FeatureUnion([
        ('onehot_pipeline', Pipeline([
            ('onehot_union', FeatureUnion([
                ('hour', DateTimeExtractor('hour')),
                ('month', DateTimeExtractor('month')), 
                ('dayofweek', DateTimeExtractor('dayofweek')), 
                ('holiday', HolidaySelector())
            ])),
            ('onehot_encoder', OneHotEncoder(sparse=False))
        ])),
        ('float_pipeline', Pipeline([
            ('float_union', FeatureUnion([
                ('year', DateTimeExtractor('year')),
                ('floats', ColumnSelector(['precip_prob', 'temperature', 'humidity', 'wind_speed', 'rides'])),
            ])),
            ('scaler', StandardScaler())
        ]))
    ]))
])


In [7]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    all_pipeline.fit(X.iloc[0:train_cut, :])
    all_X = all_pipeline.transform(X)
  
    time_pipeline.fit(X.iloc[0:train_cut, :])
    time_X = time_pipeline.transform(X)

In [8]:
from sklearn.externals import joblib

joblib.dump(all_pipeline, 'all_pipeline.pkl', compress=1)
joblib.dump(time_pipeline, 'time_pipeline.pkl', compress=1)

['time_pipeline.pkl']

In [9]:
def create_datasets(time_X, all_X, start, stop):
    ONE_DAY = 24
    lookback = 28*ONE_DAY 
    lag = 2*ONE_DAY 
    
    if start < lookback + lag:
        start = lookback + lag

    input_time_X = np.zeros((stop-start, lookback//ONE_DAY, len(time_X[0])))
    input_all_X = np.zeros((stop-start, len(all_X[0])))
    
    for i in range(start, stop):
        input_time_X[i-start] = time_X[i-lookback-lag:i-lag:ONE_DAY]
        input_all_X[i-start] = all_X[i]
    
    return input_time_X, input_all_X

input_time_X, input_all_X = create_datasets(time_X, all_X, 0, train_cut)
v_input_time_X, v_input_all_X = create_datasets(time_X, all_X, train_cut, validate_cut)

In [10]:
temporal = Input(shape=(len(input_time_X[0]), len(input_time_X[0][0])), name='temporal')
temporal1 = GRU(32, dropout=0.5, recurrent_dropout=0.5, name='temporal_gru_1')(temporal)
temporal1 = Dense(64, name='temporal_dense_1')(temporal1)
temporal1 = Dropout(0.5, name='temporal_dropout_1')(temporal1)


weather = Input(shape=(len(input_all_X[0]), ), name='all')
weather1 = Dense(64, activation='relu', name='all_dense_1')(weather)
weather1 = Dropout(0.5, name='all_dropout_1')(weather1)


concat = concatenate([temporal1, weather1], name='concat')
concat1 = Dense(128, activation='relu', name='concat_dense_1')(concat)
concat1 = Dropout(0.5, name='concat_dropout_1')(concat1)
concat1 = Dense(128, activation='relu', name='concat_dense_2')(concat1)
concat1 = Dropout(0.5, name='concat_dropout_2')(concat1)
output = Dense(1, name='output')(concat1)


model = Model([temporal, weather], output)
model.compile(optimizer='adam', loss='mse')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
temporal (InputLayer)           (None, 28, 51)       0                                            
__________________________________________________________________________________________________
temporal_gru_1 (GRU)            (None, 32)           8064        temporal[0][0]                   
__________________________________________________________________________________________________
all (InputLayer)                (None, 60)           0                                            
__________________________________________________________________________________________________
temporal_dense_1 (Dense)        (None, 64)           2112        temporal_gru_1[0][0]             
__________________________________________________________________________________________________
all_dense_

In [11]:
from keras.utils.vis_utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [12]:
callbacks_list = [
    EarlyStopping(
        monitor='val_loss',
        patience=20
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=10
    ),
    ModelCheckpoint(
        filepath='new_model_5.h5',
        monitor='val_loss',
        save_best_only=True
    )
]

model.fit([input_time_X, input_all_X], y[30*24:train_cut].to_numpy(), epochs=100, batch_size=64,
                validation_data=([v_input_time_X, v_input_all_X], y[train_cut:validate_cut]),
                callbacks=callbacks_list)

Train on 32333 samples, validate on 4132 samples
Epoch 1/100
32333/32333 [==============================] - 20s 613us/step - loss: 819302.7414 - val_loss: 741155.4454
Epoch 2/100
32333/32333 [==============================] - 17s 524us/step - loss: 368104.1903 - val_loss: 672789.0495
Epoch 3/100
32333/32333 [==============================] - 17s 523us/step - loss: 297840.3854 - val_loss: 527269.6971
Epoch 4/100
32333/32333 [==============================] - 17s 526us/step - loss: 255298.4794 - val_loss: 443803.2124
Epoch 5/100
32333/32333 [==============================] - 17s 527us/step - loss: 233883.0458 - val_loss: 331667.1234
Epoch 6/100
32333/32333 [==============================] - 17s 524us/step - loss: 217867.2793 - val_loss: 303597.4737
Epoch 7/100
32333/32333 [==============================] - 17s 528us/step - loss: 208450.4635 - val_loss: 303188.7834
Epoch 8/100
32333/32333 [==============================] - 17s 524us/step - loss: 203613.1941 - val_loss: 278871.1286
Epoch 9

32333/32333 [==============================] - 19s 590us/step - loss: 150766.3307 - val_loss: 196857.4604
Epoch 70/100
32333/32333 [==============================] - 19s 584us/step - loss: 147194.5679 - val_loss: 193133.2690
Epoch 71/100
32333/32333 [==============================] - 19s 583us/step - loss: 144708.7711 - val_loss: 193111.2823
Epoch 72/100
32333/32333 [==============================] - 19s 587us/step - loss: 145702.7963 - val_loss: 197001.6493
Epoch 73/100
32333/32333 [==============================] - 19s 585us/step - loss: 146357.7212 - val_loss: 190549.7509
Epoch 74/100
32333/32333 [==============================] - 19s 586us/step - loss: 144147.7585 - val_loss: 190285.2359
Epoch 75/100
32333/32333 [==============================] - 19s 596us/step - loss: 144693.6224 - val_loss: 190099.2057
Epoch 76/100
32333/32333 [==============================] - 19s 584us/step - loss: 146070.0904 - val_loss: 190835.5111
Epoch 77/100
32333/32333 [==============================] - 1

In [13]:
model = load_model('new_model_5.h5')

print(model.evaluate([input_time_X, input_all_X],  y[30*24:train_cut].to_numpy()))
print(model.evaluate([v_input_time_X, v_input_all_X], y[train_cut:validate_cut].to_numpy()))

32333/32333 [==============================] - 16s 498us/step
51777.24979298675
4132/4132 [==============================] - 2s 465us/step
188623.02609964908
